# Timing experiments 1 - offsets

This notebook contains code snippets that can be used to measure clock time for each step used in offsets processing. 

We use this to pick the fastest solution to coding issues that can't be solved with plain numpy vectorization.

In [1]:
import os, glob
import time
import numpy as np

from astropy.table import Table

import rawpy

## Read prototype image

This image is used as a template for looping over all pixels in one raw frame.

In [2]:
datadir = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/'

In [3]:
file_name = os.path.join(datadir, 'DSC03770.ARW')
print(file_name)

raw = rawpy.imread(file_name)
imarray = raw.raw_image_visible.astype(float)

../astrophotography_data/MilkyWayPrettyBoy/12800/light/DSC03770.ARW


In [4]:
print(imarray.shape)
ny = imarray.shape[0]
nx = imarray.shape[1]

(2848, 4256)


## Read last table in sequence

The last table contains only stars that are present in all previous tables. Thus we can use it to drive a realistic timing test. 

In [5]:
table_list = glob.glob(datadir + '/*.offsets_table.fits')
table_list.sort()
print(table_list[-1])

offsets_table = Table.read(table_list[-1])

../astrophotography_data/MilkyWayPrettyBoy/12800/light/DSC03829.offsets_table.fits


In [6]:
offsets_table

id,xcentroid,ycentroid,sharpness,roundness1,roundness2,npix,sky,peak,flux,mag,xoffset,yoffset,xoffset_prev,yoffset_prev,ref_row,prev_row
int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64
12,4057.23428944849,37.501282627404876,0.6255534815274955,-0.13035903590695222,-0.04262950687732578,25,0.0,2109.370030539578,1.8324258551870407,-0.6575660274035364,-87.84425766157256,-5.467778520996887,-1.5299578676304009,-0.048228652179709286,12,0
17,2889.890455473514,45.52478702135995,0.41365575811250577,-0.03475474039877437,0.33711999871008413,25,0.0,1267.1553678237385,1.5858006062130152,-0.5006214488390172,-53.07390723997105,-2.8195271701421802,-1.1223671286224999,0.055332762537837255,13,1
24,2764.9591473498076,69.3942446407432,0.8639779160237718,-0.2240141949572272,0.1814554181247305,25,0.0,5569.577003735975,3.8931127957568927,-1.4757424669534744,-51.03836395298504,-2.108073380868049,-0.7514724230954926,0.08979483638633212,17,2
26,3237.0702416896474,72.55740328369315,0.7436361997061437,-0.4512663738003762,0.010104039259551839,25,0.0,2028.4382918112365,1.6438839359526491,-0.5396778788819001,-58.00155007364583,-5.0574814276426565,-1.1637822747784412,-0.24496945913968204,23,3
34,3804.1341742398067,92.0060910260806,0.338153108176713,-0.7111009629819625,-0.3465544015472982,25,0.0,2559.8367531582367,3.5412108951098458,-1.3728794793176835,-74.84073386459477,-7.079134618850162,-1.0863417229443257,-0.05994521542255882,25,4
35,3399.65956182854,98.09113609675815,0.622632157331309,-0.5901394827494575,-0.1615603643315067,25,0.0,2199.6220708847322,2.02798945111849,-0.7676642290534613,-61.31184815711276,-6.464544827042374,-1.1646165490078602,0.023392983169586046,27,5
38,1203.343683276709,114.26959278472731,0.3944858788803717,0.20854002341804606,0.21925592982713288,25,0.0,1243.766969302332,1.4527461781594255,-0.40547435395891407,-52.08108485974981,11.5417191575539,-0.9084755522446812,0.1610781148148135,26,6
43,3288.064949578804,125.96082869199668,0.482611421526518,-0.6134313405293523,-0.1402967716599808,25,0.0,1693.39199641931,1.903775514420821,-0.6990393420065626,-58.008333378351836,-6.375860230146117,-0.8558311548995334,-0.1360379640099012,37,7
55,2660.0315377864654,148.94635889712774,0.7914094328637472,-0.25957987209261507,0.10065802851943292,25,0.0,1748.3392470239655,1.1993280246636255,-0.19734495486503895,-48.826340742948105,-1.9745048481723302,-0.9260609728326017,-0.1091914416224995,44,8


## Empty loop

In [7]:
t1 = time.time()
for j in range(ny):
    for i in range(nx):
        pass
t2 = time.time() - t1
print(t2, "sec")

0.36586499214172363 sec


## Read pixels

Typically we won't need to grab the pixel value itself, since we are dealing only with pixel positions. But we will need eventually to store the X and Y offsets associated to each pixel. Thus, accessing image arrays is part of the overall method. In fact, these offset arrays are the final product of the entire offset processing step.

In [8]:
t1 = time.time()
for j in range(ny):
    for i in range(nx):
        pix = imarray[j][i]
t2 = time.time() - t1
print(t2, "sec")

2.840592861175537 sec


## Store value in array

In [9]:
pixarray = np.asarray(imarray)
t1 = time.time()
for j in range(ny):
    for i in range(nx):
        pixarray[j][i] = 0.0
t2 = time.time() - t1
print(t2, "sec")

2.6494410037994385 sec


## Locate star closest to pixel

For any given pixel, the algorithm must locate the closest star in each quadrant.

In [10]:
centroid_x = offsets_table['xcentroid'].data
centroid_y = offsets_table['ycentroid'].data

In [11]:
t1 = time.time()

# arbitrary pixel for testing
pixel_x = 2128
pixel_y = 1460

In [12]:
# differences pixel - star centroid
diff_x = pixel_x - centroid_x
diff_y = pixel_y - centroid_y

# mask to reject all stars not in first quadrant
mask_x = np.where(diff_x > 0.0, 1, 0)
mask_y = np.where(diff_y > 0.0, 1, 0)
mask = mask_x * mask_y

distance = np.sqrt((diff_x * diff_x + diff_y * diff_y)) * mask

In [13]:
mindist = np.min(distance[np.nonzero(distance)])
index = np.where(distance == mindist )[0][0]

t2 = time.time() - t1
print(t2, "sec")

0.007289886474609375 sec


In [14]:
print(mindist, index)
print(diff_x[index], diff_y[index])
print(centroid_x[index], centroid_y[index])

90.17960327671783 280
79.75334799609072 42.092331018377536
2048.2466520039093 1417.9076689816225


## Locate star closest to pixel 2 - repeat over four quadrants, over many pixels

In [15]:
gt_zero = lambda x: x > 0.0
lt_zero = lambda x: x < 0.0

def closest(diff_x, diff_y, compare_x, compare_y):
    # Compute mask that masks out everything that is outside 
    # the quadrant defined by the comparison functions
    mask_x = np.where(compare_x(diff_x), 1, 0)
    mask_y = np.where(compare_y(diff_y), 1, 0)
    mask = mask_x * mask_y

    # Get index of star at minimum distance
    distance = np.sqrt((diff_x * diff_x + diff_y * diff_y)) * mask
    if np.nonzero(distance)[0].size > 0:
        mindist = np.min(distance[np.nonzero(distance)])
        index = np.where(distance == mindist )[0][0]
        return index, mindist
    else:
        return -1, 0.0

t1 = time.time()

for i in range(0, nx, 2):
    for j in range(0, ny, 2):
        pixel_x = i
        pixel_y = j

        diff_x = pixel_x - centroid_x
        diff_y = pixel_y - centroid_y
        
        index = np.array(range(4), dtype=int)
        dist  = np.array(range(4), dtype=float)
        
        # get index and distance of the closest star, one per quadrant
        index[0], dist[0] = closest(diff_x, diff_y, gt_zero, gt_zero)
        index[1], dist[1] = closest(diff_x, diff_y, lt_zero, gt_zero)
        index[2], dist[2] = closest(diff_x, diff_y, gt_zero, lt_zero)
        index[3], dist[3] = closest(diff_x, diff_y, lt_zero, lt_zero)

        t2 = time.time() - t1
        if i % 1000 == 0 and j % 1000 == 0:
            print(i, j, index[0], index[1], index[2], index[3], t2, "sec")
            print(i, j, dist[0], dist[1], dist[2], dist[3])

print((time.time() - t1), "sec")

0 0 -1 -1 -1 6 0.00131988525390625 sec
0 0 0.0 0.0 0.0 1208.757030969063
0 1000 -1 159 -1 229 0.06025886535644531 sec
0 1000 0.0 1395.4457791870966 0.0 414.3843841270778
0 2000 -1 390 -1 444 0.11823701858520508 sec
0 2000 0.0 448.4547522915838 0.0 291.998033322789
1000 0 -1 -1 200 6 89.17554306983948 sec
1000 0 0.0 0.0 1207.696964090634 233.25135232989746
1000 1000 -1 159 200 180 89.2389919757843 sec
1000 1000 0.0 396.68665898479355 384.0802017681573 160.57642026579657
1000 2000 337 387 444 465 89.30937504768372 sec
1000 2000 506.8169468283493 354.61615459146105 710.0022576946972 479.4971076427624
2000 0 -1 -1 12 15 185.40675687789917 sec
2000 0 0.0 0.0 214.14804486391438 249.79899624361656
2000 1000 165 118 228 203 185.48018407821655 sec
2000 1000 71.99809367891362 288.1212881550183 339.7901842102872 194.96616541701877
2000 2000 414 402 441 440 185.55442309379578 sec
2000 2000 106.54454325467611 150.06511847833647 40.833239010430056 16.504412564202198
3000 0 -1 -1 1 11 287.52708315849

7 min to process one image, at every other row and column (1/4 total pixels). The empty pixels could be filled with a fast call to a scipy interpolator.

This definitely can be parallelized by dividing the image in segments. 

Test with 1/2 pixels, after parallelization.